<a href="https://colab.research.google.com/github/Golem8/Music-Genre-Classifier/blob/main/Preprocessing/data_labelling_and_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Filename:** data_labelling_and_split.ipynb

**Author:** Matthew Mao

**Description:** Sort dataset into labelled folders, then split into 3 sets. Don't forget to change your paths t

---

https://nbviewer.jupyter.org/github/mdeff/fma/blob/outputs/usage.ipynb

https://gist.github.com/drscotthawley/eb4ffb1ec4de29632403c1db396e419a

In [14]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy genres and track metadata to local runtime
!cp "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/fma_metadata/genres.csv" /content/
!cp "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/fma_metadata/tracks.csv" /content/

# Copy utils module provided along with dataset
!cp "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/utils.py" /content/

In [17]:
import os

import IPython.display as ipd
import sklearn as skl

!pip install python-dotenv
import utils

In [18]:
tracks = utils.load('/content/tracks.csv')
genres = utils.load('/content/genres.csv')

In [ ]:
ipd.display(tracks['track'].head())

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [19]:
def get_audio_path(audio_dir, track_id, slice_num, suffix):
    tid_str = '{:06d}'.format(track_id)
    if 0 < track_id < 31000:
        return os.path.join(audio_dir, "block1", tid_str + "_" + str(slice_num) + "_" + suffix + '.wav')
    if 31000 < track_id < 61000:
        return os.path.join(audio_dir, "block2", tid_str + "_" + str(slice_num) + "_" + suffix + '.wav')
    if 61000 < track_id < 91000:
        return os.path.join(audio_dir, "block3", tid_str + "_" + str(slice_num) + "_" + suffix + '.wav')
    if 91000 < track_id < 121000:
        return os.path.join(audio_dir, "block4", tid_str + "_" + str(slice_num) + "_" + suffix + '.wav')
    if 121000 < track_id < 156000:
        return os.path.join(audio_dir, "block5", tid_str + "_" + str(slice_num) + "_" + suffix + '.wav')

In [20]:
AUDIO_DIR = "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data"
if not os.path.exists(AUDIO_DIR):
    os.mkdir(AUDIO_DIR)

NEW_DIR = os.path.join(AUDIO_DIR, "labelled")
if not os.path.exists(NEW_DIR):
    os.mkdir(NEW_DIR)

small_tracks = tracks['set', 'subset'] <= 'small'
small_genres = tracks.loc[small_tracks, ('track', 'genre_top')]
augment = ["reg", "wn"]

In [ ]:
get_audio_path(AUDIO_DIR, 2, 0, "reg") # test

'/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/000002_0_reg.wav'

In [34]:
# verify all files present
for track_id, genre in small_genres.iteritems():
    for slice_num in range(14): # Each file has 0-13 slices
        for suffix in augment: # Each file has "reg" and "wn" version
            filepath = get_audio_path(AUDIO_DIR, track_id, slice_num, suffix)
            if not os.path.isfile(filepath):
                print(filepath, "does not exist!" )

/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_0_reg.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_0_wn.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_1_reg.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_1_wn.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_2_reg.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/block1/012053_2_wn.wav does not exist!
/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fu

KeyboardInterrupt: ignored

In [ ]:
# Sort songs into labelled folders inside /data/labelled/
for track_id, genre in small_genres.iteritems():
    new_path = os.path.join(NEW_DIR, genre)
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    for slice_num in range(14): # Each file has 0-13 slices
        for suffix in augment: # Each file has "reg" and "wn" version
            filepath = get_audio_path(AUDIO_DIR, track_id, slice_num, suffix)
            tid_str = '{:06d}'.format(track_id)
            filename = tid_str + "_" + str(slice_num) + "_" + suffix + '.wav'
            print("Moving", filename, "to", genre)
            os.rename(filepath, new_path + "/" + filename) # move file

In [ ]:
# For splitting into training, validation, and testing
# https://pypi.org/project/split-folders/
!pip install split-folders
import splitfolders

input_path = "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/labelled"
output_path = "/content/drive/MyDrive/Year 1 (2020-2021)/APS360 - Applied Fundamentals of Machine Learning/APS360 Team Project/data/datasets"

trainPercent = 0.7
valPercent = 0.15
testPercent = 0.15

# The group_prefix parameter will use the first 6 digits as an id to keep clips of the same song together, one song won't appear in two datasets
# The package will shuffle the data
splitfolders.ratio(input, output=output, seed=9999, ratio[trainPercent, valPercent, testPercent], group_prefix=6)